Make sure docker is running

In [4]:
import subprocess
docker_desktop_path = "C:\\Program Files\\Docker\\Docker\\resources\\bin\\docker.exe"
flask_code_dir = "C:\\Users\\guang\\OneDrive\\Desktop\\Skillsfuture\\code\\flaskapp"
result = subprocess.run([docker_desktop_path, "--version"], capture_output=True, text=True)
print(result.stdout)

Docker version 27.1.1, build 6312585



go to flask dir

In [5]:
# Change to the flaskapp directory and list files
result = subprocess.run("dir", capture_output=True, text=True, cwd=r"C:/Users/guang/OneDrive/Desktop/Skillsfuture/code/flaskapp", shell=True)
print(result.stdout)

 Volume in drive C is BOOTCAMP
 Volume Serial Number is 0246-9FF7

 Directory of C:\Users\guang\OneDrive\Desktop\Skillsfuture\code\flaskapp

14/09/2024  11:51 pm    <DIR>          .
14/09/2024  11:51 pm    <DIR>          ..
15/09/2024  12:22 am    <DIR>          .elasticbeanstalk
14/09/2024  11:51 pm               113 .gitignore
14/09/2024  06:00 pm             1,653 best_condo_price_predictor.joblib
14/09/2024  08:02 pm               556 Dockerfile
15/09/2024  07:00 am             3,055 flask_app.py
14/09/2024  06:00 pm               567 le_floor_range.joblib
14/09/2024  06:00 pm               528 le_market_segment.joblib
14/09/2024  06:00 pm               138 normalization_params.joblib
               7 File(s)          6,610 bytes
               3 Dir(s)  31,814,074,368 bytes free



Remove old container and image, rebuild it

In [30]:
# Stop the existing container if it exists
result = subprocess.run([docker_desktop_path, "stop", "my_mlapi"], capture_output=True, text=True)
print(result.stdout)

# Remove the existing container if it exists
result = subprocess.run([docker_desktop_path, "rm", "my_mlapi"], capture_output=True, text=True)
print(result.stdout)

# Remove the existing image if it exists
result = subprocess.run([docker_desktop_path, "rmi", "ml-api"], capture_output=True, text=True)
print(result.stdout)

# Build the Docker image based on updated flask app
result = subprocess.run([docker_desktop_path, "build", "-t", "ml-api", "."], capture_output=True, text=True, cwd="C:/Users/guang/OneDrive/Desktop/Skillsfuture/code/flaskapp")
print(result.stdout)
if result.stderr:
    print("Error:", result.stderr)
# Run the Docker container
result = subprocess.run([docker_desktop_path, "run", "-d", "-p", "5000:5000", "--name", "my_mlapi", "ml-api:latest"], capture_output=True, text=True)
print(result.stdout)
if result.stderr:
    print("Error:", result.stderr)


my_mlapi

my_mlapi

Untagged: ml-api:latest
Deleted: sha256:d540b96bb9958d477f1b0f60b9a3d16d5d536b6b3c3ec1390fa37ab04cb90bff



Test the API on product server

To modify sample data:  
Market Segment Classes: ['OCR']  
Floor Range Classes: ['01-05' '06-10' '11-15' '16-20']  

In [32]:
import requests
# Sample data for the request
data = {
    "market_segment": "OCR",
    "months_since_contract_date": 55,
    "months_since_commence_date": 120,
    "area": 425,
    "floor_range": "11-15"
}

# Make a POST request to the /predict endpoint
response = requests.post('http://127.0.0.1:5000/predict', json=data)

# Print the response
print(response.json())

{'predicted_price': 4469640.756883434}


Test the throttling limit of 1 requests per second and 1 concurrent request 


In [33]:
# Send inference request to cloud api

import requests
import time
# Sample data for the request
data = {
    "market_segment": "OCR",
    "months_since_contract_date": 55,
    "months_since_commence_date": 120,
    "area": 425,
    "floor_range": "11-15"
}

# Make a POST request to the /predict endpoint
# Test the throttling limit of 1 requests per second
attempts_success = 0
start = time.time()
for i in range(10):
    data['area'] = data['area'] + 1 # change data slightly each time to prevent caching
    response = requests.post('https://w48kbyj7rd.execute-api.us-west-2.amazonaws.com/prod/predict', json=data)
    # Print the response
    print(response.json())
    if 'predicted_price' in response.json().keys():
        attempts_success += 1
end = time.time()
print(f"Time taken: {end - start} seconds. total attempts: {attempts_success}. \nmax requests per second: {attempts_success/(end-start)} attempts per sec")




{'predicted_price': 3806921.0154515835}
{'message': 'Too Many Requests'}
{'message': 'Too Many Requests'}
{'predicted_price': 3832182.4465891435}
{'predicted_price': 3840602.923634996}
{'message': 'Too Many Requests'}
{'predicted_price': 3857443.8777267025}
{'predicted_price': 3865864.354772556}
{'message': 'Too Many Requests'}
{'predicted_price': 3882705.308864263}
Time taken: 6.251432418823242 seconds. total attempts: 6. 
max requests per second: 0.9597800308828146 attempts per sec


Test the API on Cloud Server deployed on AWS Elastic Beanstalk

To modify sample data:  
Market Segment Classes: ['OCR']  
Floor Range Classes:  ['01-05' '06-10' '11-15' '16-20' '21-25']

In [42]:
import requests
# Sample data for the request
data = {
    "market_segment": "OCR", # Market Segment Classes: ['OCR'] 
    "months_since_contract_date": 10, # Floor Range Classes: ['01-05' '06-10' '11-15' '16-20']  
    "months_since_commence_date": 123, # 
    "area": 165,
    "floor_range": "01-05" # Floor Range Classes:  ['01-05' '06-10' '11-15' '16-20' '21-25']
}

response = requests.post('https://w48kbyj7rd.execute-api.us-west-2.amazonaws.com/prod/predict', json=data)
# Print the response
print(response.json())

{'predicted_price': 1842838.7864513365}
